In [1]:
# import sys
# import os
# sys.dont_write_bytecode = True
# os.environ["NUMBA_DISABLE_JIT"] = "1"

In [2]:
import numpy as np
from my_stuff import MufexKeys
import requests
from quantfreedom.exchanges.mufex_exchange.mufex import Mufex
from quantfreedom.enums import DynamicOrderSettingsArrays
from quantfreedom.enums import *
from quantfreedom.helper_funcs import dos_cart_product
from quantfreedom.nb_funcs.nb_strategies.nb_strategy import (
    nb_strat_bt_create_ind,
    nb_strat_evaluate,
    nb_strat_get_current_ind_settings,
    nb_strat_get_ind_set_str,
    nb_strat_get_total_ind_settings,
)
from quantfreedom.nb_funcs.nb_simulate import nb_sim_df_backtest
from quantfreedom.sim_order_records import sim_get_or
from quantfreedom.nb_funcs.nb_sim_order_records import nb_sim_get_or


mufex_main = Mufex(
    api_key=MufexKeys.api_key,
    secret_key=MufexKeys.secret_key,
    use_test_net=False,
)

%load_ext autoreload
%autoreload 2

In [3]:
candles_np = mufex_main.get_candles(symbol="BTCUSDT", timeframe="5m", candles_to_dl=1000)

It took 00 mins and 01 seconds to download 1000 candles


In [4]:
mufex_main.set_exchange_settings(
    symbol="BTCUSDT",
    position_mode=PositionModeType.HedgeMode,
    leverage_mode=LeverageModeType.Isolated,
)
backtest_settings = BacktestSettings(
    divide_records_array_size_by=10,
    gains_pct_filter=-90,
    total_trade_filter=60,
    upside_filter=-np.inf,
)
dos_arrays = DynamicOrderSettingsArrays(
    entry_size_asset=np.array([0]),
    max_equity_risk_pct=np.array([6, 9, 12]),
    max_trades=np.array([3]),
    num_candles=np.array([100]),
    risk_account_pct_size=np.array([3]),
    risk_reward=np.array([2, 4, 6]),
    sl_based_on_add_pct=np.array([0.1, 0.5, 1., .05]),
    sl_based_on_lookback=np.array([10, 30]),
    sl_bcb_type=np.array([CandleBodyType.Low]),
    sl_to_be_cb_type=np.array([CandleBodyType.Nothing]),
    sl_to_be_when_pct=np.array([0]),
    sl_to_be_ze_type=np.array([0]),
    static_leverage=np.array([0]),
    trail_sl_bcb_type=np.array([CandleBodyType.Low]),
    trail_sl_by_pct=np.array([0.5, 1.0]),
    trail_sl_when_pct=np.array([.5, 1, 2]),
)
static_os = StaticOrderSettings(
    increase_position_type=IncreasePositionType.RiskPctAccountEntrySize,
    leverage_strategy_type=LeverageStrategyType.Dynamic,
    long_or_short=LongOrShortType.Long,
    logger_bool=True,
    pg_min_max_sl_bcb=PriceGetterType.Min,
    sl_to_be_bool=False,
    z_or_e_type=ZeroOrEntryType.Nothing,
    sl_strategy_type=StopLossStrategyType.SLBasedOnCandleBody,
    tp_strategy_type=TakeProfitStrategyType.RiskReward,
    tp_fee_type=TakeProfitFeeType.Limit,
    trail_sl_bool=True,
)
dos_cart_arrays = dos_cart_product(
    dos_arrays=dos_arrays,
)

ExchangeSettings(
    limit_fee_pct = 0.0003,
    max_leverage = 150.0,
    market_fee_pct = 0.0009,
    mmr_pct = 0.004,
    min_leverage = 1.0,
    max_asset_size = 100.0,
    min_asset_size = 0.001,
    asset_tick_step = 3,
    position_mode = 1,
    leverage_mode = 1,
    price_tick_step = 1,
    leverage_tick_step = 2,
)


In [5]:
nb_sim_df_backtest(
    backtest_settings=backtest_settings,
    candles=candles_np,
    dos_cart_arrays=dos_cart_arrays,
    evaluate=nb_strat_evaluate,
    exchange_settings=mufex_main.exchange_settings,
    get_current_ind_settings=nb_strat_get_current_ind_settings,
    get_ind_set_str=nb_strat_get_ind_set_str,
    get_total_ind_settings=nb_strat_get_total_ind_settings,
    ind_creator=nb_strat_bt_create_ind,
    nb_logger_type=NBLoggerType.Pass,
    starting_equity=1000.0,
    static_os=static_os,
)

Starting the backtest now ... and also here are some stats for your backtest.

Total indicator settings to test: 6
Total order settings to test: 432
Total combinations of settings to test: 2,592
Total candles: 1,000
Total candles to test: 2,592,000


,ind_set_idx,dos_index,total_trades,wins,losses,gains_pct,win_rate,to_the_upside,fees_paid,total_pnl,ending_eq
0,0,0,61.0,15,46,-59.71,24.59,0.0,686.704,-597.0770,402.9230
1,0,36,65.0,15,50,-58.52,23.08,0.0,752.174,-585.1775,414.8225
2,0,84,74.0,11,63,-75.87,14.86,0.0,861.117,-758.6755,241.3245
3,0,96,62.0,13,49,-74.18,20.97,0.0,667.934,-741.8033,258.1967
4,0,132,77.0,12,65,-75.70,15.58,0.0,876.964,-757.0408,242.9592
5,0,133,64.0,10,54,-64.74,15.62,0.0,735.322,-647.3857,352.6143
6,0,144,64.0,14,50,-77.93,21.88,0.0,724.716,-779.3222,220.6778
7,0,180,80.0,13,67,-85.49,16.25,0.0,880.232,-854.8666,145.1334
8,0,192,62.0,13,49,-79.02,20.97,0.0,783.544,-790.1526,209.8474
9,0,228,81.0,12,69,-85.77,14.81,0.0,964.395,-857.7128,142.2872


In [17]:
order_records_df, data = nb_sim_get_or(
    backtest_settings=backtest_settings,
    candles=candles_np,
    dos_cart_arrays=dos_cart_arrays,
    evaluate=nb_strat_evaluate,
    exchange_settings=mufex_main.exchange_settings,
    get_current_ind_settings=nb_strat_get_current_ind_settings,
    get_ind_set_str=nb_strat_get_ind_set_str,
    get_total_ind_settings=nb_strat_get_total_ind_settings,
    ind_creator=nb_strat_bt_create_ind,
    nb_logger_type=NBLoggerType.Pass,
    starting_equity=1000.0,
    static_os=static_os,
    ind_set_index=3,
    dos_index=0,
)

In [18]:
order_records_df

,ind_set_idx,or_set_idx,bar_idx,timestamp,datetime,order_status,equity,available_balance,cash_borrowed,cash_used,...,entry_size_usd,entry_price,exit_price,position_size_asset,position_size_usd,realized_pnl,sl_pct,sl_price,tp_pct,tp_price
0,3,0,138,1698152400000,2023-10-24 13:00:00,EntryFilled,1000.0000,929.1540,4995.008,70.846,...,5065.854,34563.7,NaN,0.147,5065.854,NaN,0.41,34421.1,1.3,35014.7
1,3,0,139,1698152700000,2023-10-24 13:05:00,EntryFilled,1000.0000,817.0490,13402.542,182.951,...,8590.485,34479.5,NaN,0.396,13656.339,NaN,0.32,34421.1,1.0,34876.1
2,3,0,140,1698153000000,2023-10-24 13:10:00,StopLossFilled,931.8654,931.8654,NaN,NaN,...,NaN,NaN,34421.1,NaN,NaN,-68.1346,NaN,NaN,NaN,NaN
3,3,0,141,1698153300000,2023-10-24 13:15:00,EntryFilled,931.8654,868.8590,4279.487,63.006,...,4342.493,34393.1,NaN,0.126,4342.493,NaN,0.47,34233.1,1.4,34878.0
4,3,0,142,1698153600000,2023-10-24 13:20:00,EntryFilled,931.8654,810.1070,8157.806,121.758,...,4000.077,34412.2,NaN,0.242,8342.570,NaN,0.48,34233.1,1.5,34902.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
219,3,0,984,1698406200000,2023-10-27 11:30:00,StopLossFilled,374.3290,374.3290,NaN,NaN,...,NaN,NaN,33994.5,NaN,NaN,-5.1850,NaN,NaN,NaN,NaN
220,3,0,985,1698406500000,2023-10-27 11:35:00,EntryFilled,374.3290,341.3230,2693.550,33.006,...,2726.556,34027.8,NaN,0.080,2726.556,NaN,0.22,33951.7,0.9,34343.3
221,3,0,986,1698406800000,2023-10-27 11:40:00,EntryFilled,374.3290,298.6090,6256.997,75.720,...,3639.167,33993.3,NaN,0.187,6365.723,NaN,0.19,33951.7,0.8,34291.2
222,3,0,996,1698409800000,2023-10-27 12:30:00,StopLossFilled,351.0157,351.0157,NaN,NaN,...,NaN,NaN,33951.7,NaN,NaN,-23.3133,NaN,NaN,NaN,NaN


In [ ]:
res = requests.post("http://127.0.0.1:7779/plot", json=data)

# Convert response data to json
returned_data = res.json()

print(returned_data)